# MRKL

This notebook showcases using the MRKL chain to route between tasks

This uses the example Chinook database.
To set it up follow the instructions on https://database.guide/2-sample-databases-sqlite/, placing the `.db` file in a notebooks folder at the root of this repository.

In [1]:
from langchain import LLMMathChain, OpenAI, SerpAPIChain, MRKLChain, SQLDatabase, SQLDatabaseChain
from langchain.chains.mrkl.base import ChainConfig

In [2]:
llm = OpenAI(temperature=0)
search = SerpAPIChain()
llm_math_chain = LLMMathChain(llm=llm, verbose=True)
db = SQLDatabase.from_uri("sqlite:///../../../notebooks/Chinook.db")
db_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True)
chains = [
    ChainConfig(
        action_name = "Search",
        action=search.run,
        action_description="useful for when you need to answer questions about current events"
    ),
    ChainConfig(
        action_name="Calculator",
        action=llm_math_chain.run,
        action_description="useful for when you need to answer questions about math"
    ),
    
    ChainConfig(
        action_name="FooBar DB",
        action=db_chain.run,
        action_description="useful for when you need to answer questions about FooBar. Input should be in the form of a question"
    )
]

In [3]:
mrkl = MRKLChain.from_chains(llm, chains, verbose=True)

In [4]:
mrkl.run("What is the age of Olivia Wilde's boyfriend raised to the 0.23 power?")



> Entering new chain...
What is the age of Olivia Wilde's boyfriend raised to the 0.23 power?
Thought: I need to find the age of Olivia Wilde's boyfriend
Action: Search
Action Input: "Olivia Wilde's boyfriend"
Observation: Olivia Wilde started dating Harry Styles after ending her years-long engagement to Jason Sudeikis — see their relationship timeline.
Thought: I need to find the age of Harry Styles
Action: Search
Action Input: "Harry Styles age"
Observation: 28 years
Thought: I need to calculate 28 to the 0.23 power
Action: Calculator
Action Input: 28^0.23

> Entering new chain...
28^0.23

```python
print(28**0.23)
```

Answer: 2.1520202182226886

> Finished chain.

Observation: Answer: 2.1520202182226886

Thought: I now know the final answer
Final Answer: 2.1520202182226886
> Finished chain.


'2.1520202182226886'

In [5]:
mrkl.run("Who recently released an album called 'The Storm Before the Calm' and are they in the FooBar database? If so, what albums of theirs are in the FooBar database?")



> Entering new chain...
Who recently released an album called 'The Storm Before the Calm' and are they in the FooBar database? If so, what albums of theirs are in the FooBar database?
Thought: I need to find an album called 'The Storm Before the Calm'
Action: Search
Action Input: "The Storm Before the Calm album"
Observation: The Storm Before the Calm (stylized in all lowercase) is the tenth (and eighth international) studio album by Canadian-American singer-songwriter Alanis ...
Thought: I need to check if Alanis is in the FooBar database
Action: FooBar DB
Action Input: "Does Alanis Morissette exist in the FooBar database?"

> Entering new chain...
Does Alanis Morissette exist in the FooBar database?
SQLQuery: SELECT * FROM Artist WHERE Name = 'Alanis Morissette'
SQLResult: [(4, 'Alanis Morissette')]
Answer: Yes
> Finished chain.

Observation:  Yes
Thought: I need to find out what albums of Alanis's are in the FooBar database
Action: FooBar DB
Action Input: "What albums by Alanis Mo

'The album is by Alanis Morissette and the albums in the FooBar database by her are Jagged Little Pill'